In [ ]:
!pip install flask-ngrok
!pip install pyngrok
!pip install -U flask-cors
!pip install flask-ngrok
!ngrok authtoken "225D7adRDbQhsVXdxHFBfdBy79f_6bjUDDY7kCxd79n1NkhCQ"

     |████████████████████████████████| 745 kB 5.1 MB/s 
  Created wheel for pyngrok: filename=pyngrok-5.1.0-py3-none-any.whl size=19006 sha256=7bc68e2823e9d09113120ec81d5272eb2081f1a7b789b325665c92262b71780a
  Stored in directory: /root/.cache/pip/wheels/bf/e6/af/ccf6598ecefecd44104069371795cb9b3afbcd16987f6ccfb3
Successfully built pyngrok
Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [ ]:
from flask import Flask,request
from flask_ngrok import run_with_ngrok
from flask_cors import CORS, cross_origin

from io import BytesIO
import torch
from PIL import Image
import base64
from google.colab import files

In [ ]:
app = Flask(__name__)
cors = CORS(app)
app.config['CORS_HEADERS'] = 'Content-Type'
run_with_ngrok(app)  

@app.route('/')
def hello_world():
  return '<h1>Hello, Shabid!</h1>'

@app.route("/img",methods=['POST', 'GET'])
@cross_origin()
def img():
  org=bytes_in = request.args.get('img','').replace(" ","+")

  device = "cuda" if torch.cuda.is_available() else "cpu"
  model = torch.hub.load("bryandlee/animegan2-pytorch:main", "generator", device=device).eval()
  face2paint = torch.hub.load("bryandlee/animegan2-pytorch:main", "face2paint", device=device)
  image_format="png"
  print(type(bytes_in))
  bytes_in = base64.b64encode(bytes(bytes_in, 'utf-8'))
  bytes_in1=base64.decodebytes(bytes_in)
  with open('decoded_image.png', 'wb') as file_to_save:
      decoded_image_data = base64.decodebytes(bytes_in1)
      file_to_save.write(decoded_image_data)


  file = open("decoded_image.png","rb")
  bytes_in1 = file.read()
  file.close()

  im_in = Image.open(BytesIO(bytes_in1)).convert("RGB")
  im_out = face2paint(model, im_in, side_by_side=False)
  buffer_out = BytesIO()
  im_out.save(buffer_out, format=image_format)
  bytes_out = buffer_out.getvalue()
  bs64_out = base64.b64encode(bytes_out)
  print(bs64_out)
  print(str(bs64_out).replace(" ","+")[2:-1])
  return str(bs64_out).replace(" ","+")[2:-1]

if __name__ == '__main__':
  app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://75a4-34-73-242-202.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [21/Dec/2021 10:10:41] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [21/Dec/2021 10:10:42] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [21/Dec/2021 10:10:43] "GET / HTTP/1.1" 200 -
Downloading: "https://github.com/bryandlee/animegan2-pytorch/archive/main.zip" to /root/.cache/torch/hub/main.zip
Downloading: "https://github.com/bryandlee/animegan2-pytorch/archive/main.zip" to /root/.cache/torch/hub/main.zip
Downloading: "https://github.com/bryandlee/animegan2-pytorch/raw/main/weights/face_paint_512_v2.pt" to /root/.cache/torch/hub/checkpoints/face_paint_512_v2.pt


  0%|          | 0.00/8.20M [00:00<?, ?B/s]

...

<class 'str'>


127.0.0.1 - - [21/Dec/2021 10:11:18] "GET /img?img=/9j/4AAQSkZJRgABAQAAAQABAAD/2wCEAAoHCBYWFRgWFRYYGRgaGhkcHBocGhgaGhwaGhoaGhgaGRgcIS4lHCErIRgaJjgmKy8xNTU1GiQ7QDs0Py40NTEBDAwMEA8QHxISHjQrJSs0NDQ0NDQ0NDQ0NDQ0NDQ0NDQ0NDQ0NDQ0NDQ0NDQ0NDQ0NDQ0NDQ0NDQ0NDQ0NDQ0NP/AABEIAOEA4QMBIgACEQEDEQH/xAAcAAACAgMBAQAAAAAAAAAAAAAEBQMGAAIHAQj/xAA+EAACAQIDBQYDBgQGAgMAAAABAgADEQQSIQUGMUFRImFxgZHwE6GxBxQyUsHRFUJi4RYjcpKy8TPCF4Ki/8QAGQEAAwEBAQAAAAAAAAAAAAAAAQIDAAQF/8QAJBEAAwEAAgIBBQEBAQAAAAAAAAECERIhAzEEEyIyQVFhcUL/2gAMAwEAAhEDEQA/AJcLVzRgiRTsUXliSnLuk/RyLxteyNac3VIQFmy2i6OpPUUzexk1FhCUQRdDhWttY5aeh4hc552W+UacyW0A6+dqbtXbTFyubLpy/lUDM1upOgv3d5g/2g7ZAxlRBwQ0x4laalbn+kvUI7zKnUxJYkk63A8QTeaqxFJne2PFxhYhADlsXIvqdRfMeZPDrpC6ePzMxY6te5/pBtYd0rtCqcxI4kEf8bfrNWqm/lbyknX9KqQnHbRuXblcADyMjwtB6nEhV72Av48z4aQCrfh/Vf0uP1ki4k2vy4269B9T5TJmaH9GhTXV6otbQKCbj+nUWHoIY2IpLwUg8bH8R6E6sR52lQTEnVydeXj18pjYo2tzPu/jNopbHx1Em5W7c9VU/IXPrF+0sTRLBSi8AdWNzcdb359Yu2bhLkMT77v3MYVsPSLdtdT/ADA3v5q4PrDoSAbORvwIMutyHa49biLMZg2QXGo66aeNpasGopg9g

b'...


127.0.0.1 - - [21/Dec/2021 10:11:19] "GET /img?img=/9j/4AAQSkZJRgABAQAAAQABAAD/2wCEAAoHCBYWFRgWFRYYGRgaGhkcHBocGhgaGhwaGhoaGhgaGRgcIS4lHCErIRgaJjgmKy8xNTU1GiQ7QDs0Py40NTEBDAwMEA8QHxISHjQrJSs0NDQ0NDQ0NDQ0NDQ0NDQ0NDQ0NDQ0NDQ0NDQ0NDQ0NDQ0NDQ0NDQ0NDQ0NDQ0NDQ0NP/AABEIAOEA4QMBIgACEQEDEQH/xAAcAAACAgMBAQAAAAAAAAAAAAAEBQMGAAIHAQj/xAA+EAACAQIDBQYDBgQGAgMAAAABAgADEQQSIQUGMUFRImFxgZHwE6GxBxQyUsHRFUJi4RYjcpKy8TPCF4Ki/8QAGQEAAwEBAQAAAAAAAAAAAAAAAQIDAAQF/8QAJBEAAwEAAgIBBQEBAQAAAAAAAAECERIhAzEEEyIyQVFhcUL/2gAMAwEAAhEDEQA/AJcLVzRgiRTsUXliSnLuk/RyLxteyNac3VIQFmy2i6OpPUUzexk1FhCUQRdDhWttY5aeh4hc552W+UacyW0A6+dqbtXbTFyubLpy/lUDM1upOgv3d5g/2g7ZAxlRBwQ0x4laalbn+kvUI7zKnUxJYkk63A8QTeaqxFJne2PFxhYhADlsXIvqdRfMeZPDrpC6ePzMxY6te5/pBtYd0rtCqcxI4kEf8bfrNWqm/lbyknX9KqQnHbRuXblcADyMjwtB6nEhV72Av48z4aQCrfh/Vf0uP1ki4k2vy4269B9T5TJmaH9GhTXV6otbQKCbj+nUWHoIY2IpLwUg8bH8R6E6sR52lQTEnVydeXj18pjYo2tzPu/jNopbHx1Em5W7c9VU/IXPrF+0sTRLBSi8AdWNzcdb359Yu2bhLkMT77v3MYVsPSLdtdT/ADA3v5q4PrDoSAbORvwIMutyHa49biLMZg2QXGo66aeNpasGopg9g

b'...
